<a href="https://colab.research.google.com/github/jcburda/portfolio/blob/main/HokeyPoker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
# Poker Monte Carlo Simulator in Google Colab
# Non-Cumulative & Cumulative Grids, All Color-Coded
# Unconditional 2-hole hardcoded

import random
import itertools
from collections import Counter
import ipywidgets as widgets
from IPython.display import display, clear_output
from tqdm.notebook import tqdm
import time

# ======= Poker Helpers =======
RANKS = '23456789TJQKA'
SUITS = 'CDHS'
HAND_RANKS = [
    "Royal Flush", "Straight Flush", "Four of a Kind", "Full House", "Flush",
    "Straight", "Three of a Kind", "Two Pair", "One Pair", "High Card"
]

def hand_rank(hand):
    ranks = sorted([RANKS.index(c[0]) for c in hand], reverse=True)
    suits = [c[1] for c in hand]
    counts = Counter(ranks)
    values_by_count = sorted(counts.items(), key=lambda x: (-x[1], -x[0]))
    is_flush = len(set(suits)) == 1
    is_straight = len(counts) == 5 and ranks[0] - ranks[-1] == 4
    if set(ranks) == {12,0,1,2,3}:  # A2345
        is_straight = True
        ranks = [3,2,1,0,-1]
    if is_flush and ranks == [12,11,10,9,8]:
        return 0
    if is_flush and is_straight:
        return 1
    if values_by_count[0][1] == 4:
        return 2
    if values_by_count[0][1] == 3 and values_by_count[1][1] == 2:
        return 3
    if is_flush:
        return 4
    if is_straight:
        return 5
    if values_by_count[0][1] == 3:
        return 6
    if values_by_count[0][1] == 2 and values_by_count[1][1] == 2:
        return 7
    if values_by_count[0][1] == 2:
        return 8
    return 9

def best_hand(hole, community):
    all_combos = itertools.combinations(hole + community, 5)
    ranks = (hand_rank(combo) for combo in all_combos)
    return min(ranks)

# ======= Monte Carlo =======
def monte_carlo_poker(n_hole, community_cards, num_samples, pbar):
    counts = Counter()
    deck = [r+s for r in RANKS for s in SUITS]

    for _ in range(num_samples):
        available_deck = deck.copy()
        known_community = community_cards.copy() if community_cards else []
        for c in known_community:
            if c in available_deck:
                available_deck.remove(c)
        hole = random.sample(available_deck, n_hole)
        for c in hole:
            if c in available_deck:
                available_deck.remove(c)
        needed_community = 5 - len(known_community)
        sampled_community = random.sample(available_deck, needed_community)
        full_community = known_community + sampled_community
        rank = best_hand(hole, full_community)
        counts[rank] += 1
        pbar.update(1)

    return {i: counts.get(i,0)/num_samples*100 for i in range(len(HAND_RANKS))}

def cumulative_probs(prob_dict):
    cum = {}
    total = 0
    # Sum from best hand down to current hand
    for i in range(len(HAND_RANKS)):
        total += prob_dict[i]
        cum[i] = total
    return cum

# ======= ASCII Grid Builder =======
def color_code(pct):
    if 0 < pct < 5:
        return "\033[91m"  # red
    elif 5 < pct < 25:
        return "\033[93m"  # yellow
    else:
        return "\033[92m"  # green

def build_grid(prob_dicts, title):
    col_width = 18
    percent_width = 8
    header = "-"*73
    title_line = f"| {title:<{col_width}} | {'2 Hole':<{percent_width}} | {'3 Hole':<{percent_width}} | {'4 Hole':<{percent_width}} | {'5 Hole':<{percent_width}} |"
    lines = [header, title_line, header]

    for i, hand_name in enumerate(HAND_RANKS):
        row = f"| {hand_name:<{col_width}} "
        for probs in prob_dicts:
            pct = probs.get(i,0)
            color = color_code(pct)
            row += f"| {color}{pct:>{percent_width}.1f}\033[0m "
        row += "|"
        lines.append(row)
    lines.append(header)
    return "\n".join(lines)

# ======= Widgets =======
sample_input = widgets.IntText(value=5000, description='Samples:')
community_input = widgets.Text(value='AS,KS,QS,2D,3C', description='Community:')
run_button = widgets.Button(description="Run Simulation", button_style='success')
output_area = widgets.Output()
progress_label = widgets.Label(value="Waiting to start simulation...")

display(sample_input, community_input, run_button, progress_label, output_area)

# ======= Callback with Hardcoded 2-Hole Unconditional =======
def run_simulation_callback(b):
    with output_area:
        clear_output()
        start_time = time.time()
        num_samples = sample_input.value
        community_cards = [c.strip().upper() for c in community_input.value.split(",") if c.strip()] or None

        total_iterations = num_samples * 4 * 2
        pbar = tqdm(total=total_iterations, desc="Monte Carlo Simulation")

        # Precomputed exact unconditional 2-hole probabilities (percent)
        uncond_2hole_exact = {
            0: 0.003,  # Royal Flush
            1: 0.027,  # Straight Flush
            2: 0.168,  # Four of a Kind
            3: 2.60,   # Full House
            4: 3.03,   # Flush
            5: 4.62,   # Straight
            6: 4.83,   # Three of a Kind
            7: 23.5,   # Two Pair
            8: 43.8,   # One Pair
            9: 17.7    # High Card
        }
        uncond_2hole_cum = cumulative_probs(uncond_2hole_exact)

        # Unconditional
        probs_uncond_noncum = [uncond_2hole_exact]  # first column hardcoded
        probs_uncond_cum = [uncond_2hole_cum]       # first column hardcoded
        for n in range(3,6):
            progress_label.value = f"Running UNCONDITIONAL simulation for {n}-hole cards..."
            mc_result = monte_carlo_poker(n, None, num_samples, pbar)
            probs_uncond_noncum.append(mc_result)
            probs_uncond_cum.append(cumulative_probs(mc_result))

        # Conditional
        probs_cond_noncum = []
        probs_cond_cum = []
        for n in range(2,6):
            progress_label.value = f"Running CONDITIONAL simulation for {n}-hole cards..."
            mc_result = monte_carlo_poker(n, community_cards, num_samples, pbar)
            probs_cond_noncum.append(mc_result)
            probs_cond_cum.append(cumulative_probs(mc_result))

        pbar.close()
        progress_label.value = "Simulation complete!"

        print("\n------------------")
        print("NON-CUMULATIVE")
        print("------------------")
        print("\nUNCONDITIONAL:")
        print(build_grid(probs_uncond_noncum, "Hand Type"))
        print("\nCONDITIONAL:")
        print(build_grid(probs_cond_noncum, "Hand Type"))
        print("\n------------------")
        print("CUMULATIVE")
        print("------------------")
        print("\nUNCONDITIONAL:")
        print(build_grid(probs_uncond_cum, "Hand Type"))
        print("\nCONDITIONAL:")
        print(build_grid(probs_cond_cum, "Hand Type"))

        elapsed = time.time() - start_time
        print(f"\nSimulation runtime: {elapsed:.2f} seconds")

run_button.on_click(run_simulation_callback)


IntText(value=5000, description='Samples:')

Text(value='AS,KS,QS,2D,3C', description='Community:')

Button(button_style='success', description='Run Simulation', style=ButtonStyle())

Label(value='Waiting to start simulation...')

Output()